In [24]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
%matplotlib inline            
import seaborn as sns


In [ ]:
x = pd.read_csv("../input/train_folds.csv")
df_test = pd.read_csv("../input/test.csv")
sample_submission = pd.read_csv("../input/sample_submission.csv")

useful_features = [c for c in df.columns if c not in ("id", "target", "kfold")]
object_cols = [col for col in useful_features if 'cat' in col]
numerical_cols = [col for col in useful_features if 'cont' in col]
df_test = df_test[useful_features]

In [31]:




final_predictions = []
scores=[]
for fold in range(5):
    xtrain =  df[df.kfold != fold].reset_index(drop=True)
    xvalid = df[df.kfold == fold].reset_index(drop=True)
    xtest = df_test.copy()

    ytrain = xtrain.target
    yvalid = xvalid.target
    
    xtrain = xtrain[useful_features]
    xvalid = xvalid[useful_features]
    
    ordinal_encoder = preprocessing.OrdinalEncoder()
    xtrain[object_cols] = ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols] = ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols] = ordinal_encoder.transform(xtest[object_cols])
    
    
    # standarization
    scaler = preprocessing.StandardScaler()
    xtrain[numerical_cols] = scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols] = scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols] = scaler.transform(xtest[numerical_cols])
    
    model = XGBRegressor(random_state=fold, 
                         n_jobs=-1,
                         tree_method='gpu_hist',
                         n_estimators= 1500,                         
                         learning_rate= 0.06,
                         subsample= 0.7,
                         max_depth= 5,                         colsample_bytree= 0.5,
                         reg_alpha = 335eval_metric='rmse')
    model.fit(xtrain, ytrain)
    preds_valid = model.predict(xvalid)
    test_preds = model.predict(xtest)
    final_predictions.append(test_preds)
    rmse = mean_squared_error(yvalid, preds_valid, squared=False)
    print(fold,rmse)
    scores.append(rmse)
    
print (np.mean(scores),np.std(scores))
.

0 0.7182744120535692
1 0.7184895002677275
2 0.7206877619723528
3 0.7201927749471806
4 0.7191879778581044
0.7193664854197869 0.0009404380102009376


# 0.7192209564740882 0.0009130976597296758

In [ ]:
# Model hyperparameters
xgb_params = {'n_estimators': 500,
              'learning_rate': 0.5,
              'subsample': 0.926,
              'colsample_bytree': 0.84,
              'max_depth': 2,
              'booster': 'gbtree', 
              'reg_lambda': 35.1,
              'reg_alpha': 34.9,
              'random_state': 42,
              'n_jobs': 4}

In [ ]:
    model = XGBRegressor(random_state=fold, 
                         n_jobs=-1,
                         n_estimators= 8000,
                         tree_method='gpu_hist',
                         learning_rate= 0.009,
                         subsample= 0.9,
                         max_depth= 5,
                         colsample_bytree= 0.5,
                         reg_alpha = 40,eval_metric='rmse')
    model.fit(xtrain, ytrain)

In [9]:
preds = np.mean(np.column_stack(final_predictions), axis=1)

In [10]:
sample_submission.target = preds
sample_submission.to_csv("submission.csv", index=False)